# Clase 13: Detección de Anomalías y Visualización en Baja Dimensionalidad

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**




## Objetivos de la Clase

- Entender qué es un outlier y explorar distintas téncicas para encontrarlos.
- Explorar técnicas para visualizar datos en baja dimensionalidad.

## Detección y manejo de Anomalías

Una anomalía (*outlier* en ingles) es un dato significativamente distinto a los demás. Se puede considerar como una observación cuya desviación del conjunto de datos permite establecer la hipótesis, de que su generación fue obtenida por un mecanismo distinto al principal en la modelación de un fenómeno.

![Outlier](../../recursos/2023-01/16-Detección-anomalias/outliers.gif)

> **Pregunta**: ¿Por qué debería preocuparme por estos valores?

Las anomalías contienen por tanto información sobre características anormales de las entidades y esquemas que impactan el proceso generativo de los datos. Reconocer estas observaciones permite:
- Desde el punto de vista teórico, mejorar el entendimiento de los problemas modelados. 
- Desde el punto de vista práctico, permite mejorar procesos de adquisición de datos y presición de modelos.


Antes de continuar, es necesario hacer una distinción entre términos:

- **Detección de Outliers** : Detectamos anomalías sobre los datos que estamos analizando o sobre los datos de entrenamiento del modelo.

- **Novelty Detection**: Cuando detectamos outliers sobre **datos nuevos**.


## Tipos de Outliers

- **Univariados**: Solo en una característica de los datos.
- **Multivariados**: Outlier al combinar más de una característica de los datos.
   

## Origen de los Outliers


> **Pregunta ❓**: ¿Cuáles son las posibles fuentes de Outliers?


- Errores al transcribir los Datos.
- Errores de Medición.
- Errores Experimentales.
- Errores del preprocesamiento.
- Al extraer o mezclar datos que no son compatibles entre si.
- Naturales.

## Dataset de Hoy: Wine Quality 🍷

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/wine.jpg" style="width: 50%;">
</div>

<center> Fuente de la imagen: https://www.baysidegroup.com.au/blog/could-native-grapes-be-the-future-of-australian-wine/</center>

Wine Quality es un dataset de características que describen vinos portugeses de la variedad "Vinho Verde".

Atributos:

    1 - fixed acidity
    2 - volatile acidity
    3 - citric acid
    4 - residual sugar
    5 - chlorides
    6 - free sulfur dioxide
    7 - total sulfur dioxide
    8 - density
    9 - pH
    10 - sulphates
    11 - alcohol
    
Output: Calidad subjetiva del vino.

    12 - quality (score between 0 and 10)

In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../../recursos/2023-01/16-Detección-anomalias/wineQualityReds.csv", index_col=0)
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [2]:
df.describe()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [3]:
for col in df:
    fig = px.histogram(df, x=col, marginal="box", title = 'Histogram of ' + col)
    fig.show()

## Métodos de Manejo de Anomalías Univariados


#### Desviación Estándar

Si se estima que la variable a estudiar se distribuye de manera normal, entonces el 95% de los datos se encuentra a 2 desviaciones estándar de la media, mientras que el 99.7% se encuentra dentro de 3 desviaciones estándar. Basándose en esto, se puede considerar que cualquier punto fuera de 3 desviaciones estándar de la media como candidato a anomalía. Una forma más flexible de estimar anomalías usando el principio de normalidad, es por medio de z-scores. 


$$\text{z-score} = \frac{x_i - \overline{x}}{s}$$


<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/norm.png" style="width: 75%;">
</div>

In [4]:
import plotly.figure_factory as ff

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(
    [df["fixed.acidity"].values], ["fixed.acidity"], curve_type="normal", show_rug=False
)
fig.show()

In [5]:
from scipy.stats import zscore

z_scores = zscore(df["fixed.acidity"])
z_scores

1      -0.528360
2      -0.298547
3      -0.298547
4       1.654856
5      -0.528360
          ...   
1595   -1.217796
1596   -1.390155
1597   -1.160343
1598   -1.390155
1599   -1.332702
Name: fixed.acidity, Length: 1599, dtype: float64

In [6]:
# agregamos los zscores al df
df["fixed.acidity_zscores"] = z_scores
df

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,fixed.acidity_zscores
1,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,-0.528360
2,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,-0.298547
3,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,-0.298547
4,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1.654856
5,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,-0.528360
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,-1.217796
1596,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,-1.390155
1597,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,-1.160343
1598,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,-1.390155


In [7]:
df["fixed.acidity_zscores"].abs() > 3

1       False
2       False
3       False
4       False
5       False
        ...  
1595    False
1596    False
1597    False
1598    False
1599    False
Name: fixed.acidity_zscores, Length: 1599, dtype: bool

In [8]:
# agregamos un booleano por cada fila que indica si la observación está
# a 3 o más distribuciones estandar entonces es booleano.
df["fixed.acidity_outlier"] = df["fixed.acidity_zscores"].abs() > 3

df[["fixed.acidity", "fixed.acidity_zscores", "fixed.acidity_outlier"]].head(10)

,fixed.acidity,fixed.acidity_zscores,fixed.acidity_outlier
1,7.4,-0.528360,False
2,7.8,-0.298547,False
3,7.8,-0.298547,False
4,11.2,1.654856,False
5,7.4,-0.528360,False
6,7.4,-0.528360,False
7,7.9,-0.241094,False
8,7.3,-0.585813,False
9,7.8,-0.298547,False
10,7.5,-0.470907,False


In [9]:
df.loc[
    df["fixed.acidity_outlier"], # condicion: fixed.acidity_outlier == True
    ["fixed.acidity", "fixed.acidity_zscores", "fixed.acidity_outlier"],
]

,fixed.acidity,fixed.acidity_zscores,fixed.acidity_outlier
244,15.0,3.838072,True
245,15.0,3.838072,True
348,13.8,3.148635,True
375,14.0,3.263541,True
382,13.7,3.091182,True
392,13.7,3.091182,True
443,15.6,4.182790,True
545,14.3,3.435900,True
555,15.5,4.125337,True
556,15.5,4.125337,True


In [10]:
fig = px.histogram(df, x="fixed.acidity", color="fixed.acidity_outlier")
fig.update_layout(showlegend=False)

Si se va a estudiar una columna con ouliers mediante este método, es coveniente hacer un test de normalidad. Si la variable no cumple con la hipótesis, es posible preprocesarla usando el método de box-cox, Yeo-Johnson o Inter Quantilico. Se recomienda este último por ser robusto a outliers. 


#### IQR: Rango Intercuartilico

El rango intercuartílico (**IQR**) se utiliza para medir la dispersión de los datos. 
Para obtenerlo: 

Los datos se separan en 4 grupo de (casi) igual tamaño.
El IQR se calcula como la diferencia entre el primer cuartil $Q1$ (25%) y el tercero $Q3$ (75%) : $IQR = Q3 - Q1$.

<div align='center'>
    <img src='../../recursos/2023-01/16-Detección-anomalias/iqr.png'/>
    <p> 
        Fuente: 
        <a href='https://panmath7.files.wordpress.com/2017/04/how_to_find_iqr_boxplot_image.jpg'>How to find a IQR boxplot image
        </a> 
    </p>
</div>

In [11]:
df["total.sulfur.dioxide"].describe()

count    1599.000000
mean       46.467792
std        32.895324
min         6.000000
25%        22.000000
50%        38.000000
75%        62.000000
max       289.000000
Name: total.sulfur.dioxide, dtype: float64

In [12]:
px.histogram(df, "total.sulfur.dioxide", marginal="box")

In [13]:
desc = df["total.sulfur.dioxide"].describe()
desc

count    1599.000000
mean       46.467792
std        32.895324
min         6.000000
25%        22.000000
50%        38.000000
75%        62.000000
max       289.000000
Name: total.sulfur.dioxide, dtype: float64

In [14]:
iqr = desc["75%"] - desc["25%"]
iqr

40.0

Luego, los valores que estén en $25\% - (IQR * 1.5)$

In [15]:
cota_inf = desc["25%"] - iqr * 1.5
cota_inf

-38.0

Y los valores que estén sobre $75\% + (IQR * 1.5)$

In [16]:
cota_sup = desc["75%"] + iqr * 1.5
cota_sup

122.0

Pueden ser considerados como outliers

In [17]:
df["total.sulfur.dioxide_outlier"] = (
    df["total.sulfur.dioxide"] > cota_sup) | (
    df["total.sulfur.dioxide"] < cota_inf
)

In [18]:
import plotly.express as px

fig1 = px.box(df, x="total.sulfur.dioxide", height=200)
fig1.show()

fig2 = px.histogram(
    df,
    x="total.sulfur.dioxide",
    color="total.sulfur.dioxide_outlier",
    title="total.sulfur.dioxide<br><sup>Rojo = outlier</sup>",
)
fig2.update_layout(showlegend=False)
fig2.show()

En el gráfico de caja, vemos que los outliers están sobre y bajo las lineas rectas, cada una representa Q1 - 1.5 x IQR (linea inferior) y Q3 + 1.5 x IQR (linea superior) Los valores dentro de la caja corresponden al IQR y la linea central es la mediana de los datos.

> **Pregunta:** ¿Por qué 1.5 y no otro número?

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/iqr_origen.png" style="width: 50%;">
</div>

## Reducción de Dimensionalidad para la Visualización 

Como hemos visto en las clases anteriores, todos las observaciones se componen de muchas variables/características distintas. Mientras más características se tengan, más complejo se torna el análisis exploratorio de datos, la detección de outliers y la creación de modelos predictivos.



In [19]:
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,fixed.acidity_zscores,fixed.acidity_outlier,total.sulfur.dioxide_outlier
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,-0.528360,False,False
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,-0.298547,False,False
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,-0.298547,False,False
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1.654856,False,False
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,-0.528360,False,False




Por lo tanto, sería de mucha utilidad contar con mecanismos para reducir la cantidad de características. Existen 2 enfoques: 

- **Selección de Atributos Relevantes**
- **Reducción de Dimensionalidad.**

Ambos serán vistos con mayor profundidad más adelante. 

Sin embargo, existe un par de métodos de reducción de dimensionalidad que nos pueden ser particularmente útiles para la tarea que estamos resolviendo en este momento (detección de outliers) y en general, para crear una idea de como se distribuyen los datos al considerar todas las características.


Estos son los **métodos de reducción de dimensionalidad para visualización**. Estas son técnicas que nos permiten representar cada observación (un vector de alta dimensionalidad) en un vector de dos o tres dimensiones (mucho más sencillo para graficar).


**Paréntesis: Manifold learning**  


> Manifold learning, also called nonlinear dimensionality reduction, is an approach to non-linear dimensionality reduction. Algorithms for this task are based on the idea that the dimensionality of many data sets is only artificially high.

### t-distributed Stochastic Neighbor Embedding (t-SNE)


La idea detrás de este método es proveer un método efectivo de reducción de dimensionalidad para visualizar un dataset complejo. El objetivo es conservar la mayor parte de la información en la dimensión baja a la vez que permita visualizar clusters y estructura general de los datos.

Este método conserva la localidad de las observaciones: Es decir, las observaciones similares queden en vectores cercanos y observaciones distintas en vectores lejanos (cercanos y lejanos en función de sus distancias).


Sin embargo, el método posee un par de desventajas:

- Es computacionalmente caro. Puede tomar horas transformar más de un millon de datos.
- Es estocástico: diferentes ejecuciones entregan distintas proyecciones.
- Las relaciones entre distancias globales no se preservan correctamente.

> **Nota**: Es importante tener a la misma escala todos los datos.

<div style="text-align: center;">
    <img src="https://miro.medium.com/max/800/1*lKLB_1aghhnxQjMQziEyGQ.gif" style="width: 50%;">
</div>

<center>Fuente de la animación: https://www.oreilly.com/content/an-illustrated-introduction-to-the-t-sne-algorithm/</center>


No veremos la implementación de este método en clases, pero pueden encontrar una muy buena referencia en el siguiente video: https://www.youtube.com/watch?v=NEaUSP4YerM

Ver también: **https://projector.tensorflow.org/**

In [20]:
features = df.drop(
    columns=[
        "quality",
        "fixed.acidity_zscores",
        "fixed.acidity_outlier",
        "total.sulfur.dioxide_outlier",
    ]
)
quality = df.loc[:, ["quality"]]

features.head(3)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8


In [21]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

# Notar que hacemos un escalamiento previo!!
scaled_features = MinMaxScaler().fit_transform(features)

# Creamos una instancia de tsne
tsne = TSNE(n_components=2, random_state=42)

# Transformamos los datos
wine_features_tsne_embedded = tsne.fit_transform(scaled_features)
wine_features_tsne_embedded

array([[-41.049725, -16.692968],
       [-53.906433, -16.810059],
       [-30.629326, -22.942598],
       ...,
       [-19.131292,  24.774824],
       [-16.300737,  23.55727 ],
       [  5.539513,  24.342754]], dtype=float32)

In [22]:
# Noten que guardamos la transformación en df y no en features.
df["x_tsne"] = wine_features_tsne_embedded[:, 0]
df["y_tsne"] = wine_features_tsne_embedded[:, 1]

In [23]:
px.scatter(df, x="x_tsne", y="y_tsne")

In [24]:
px.scatter(df, x="x_tsne", y="y_tsne", color="pH")

In [25]:
px.scatter(df, x="x_tsne", y="y_tsne", color="alcohol")

In [26]:
px.scatter(df, x="x_tsne", y="y_tsne", color="residual.sugar")

In [27]:
px.scatter(df, x="x_tsne", y="y_tsne", color="quality")

In [28]:
px.scatter(df, x="x_tsne", y="y_tsne", color="fixed.acidity")

In [29]:
px.scatter(df, x="x_tsne", y="y_tsne", color="fixed.acidity_outlier")

### Uniform Manifold Approximation and Projection (UMAP)

Relativamente novedoso método de reducción de dimensionalidad. Puede ser usado tanto para visualización como para reducción de dimensionalidad para modelos predictivos. 

Presenta varias mejoras que con respecto a TSNE.  

- Es más rápido y acepta una mayor cantidad de datos. 
- Conserva las distancias globales entre puntos.

In [30]:
# hay que instalarlo aparte.
!pip install umap-learn

In [31]:
import umap

# Notar que hacemos un escalamiento previo!!
scaled_features = MinMaxScaler().fit_transform(features)

umap = umap.UMAP()

wine_features_umap_embedded = umap.fit_transform(scaled_features)
wine_features_umap_embedded

array([[12.319527 ,  3.8665793],
       [10.983167 ,  4.0145774],
       [11.06023  ,  3.648147 ],
       ...,
       [ 9.597175 ,  7.1312537],
       [ 9.6741   ,  6.991527 ],
       [ 6.5303555,  7.506018 ]], dtype=float32)

In [32]:
# Noten que guardamos la transformación en df y no en features.
df["x_umap"] = wine_features_umap_embedded[:, 0]
df["y_umap"] = wine_features_umap_embedded[:, 1]

In [33]:
from plotly.subplots import make_subplots

fig1 = px.scatter(
    df,
    x="x_umap",
    y="y_umap",
)

fig2 = px.scatter(
    df,
    x="x_tsne",
    y="y_tsne",
)

# izquierda: UMAP
# derecha: TSNE
fig = (
    make_subplots(rows=1, cols=2)
    .add_trace(fig1.data[0], row=1, col=1)
    .add_trace(fig2.data[0], row=1, col=2)
    .update_layout(height=400, title_text="Comparación UMAP - TSNE")
    .show()
)

In [34]:
px.scatter(
    df,
    x="x_umap",
    y="y_umap",
    color="quality",
)

In [35]:
px.scatter(df, x="x_umap", y="y_umap", color="fixed.acidity_outlier")

In [36]:
px.scatter(df, x="x_umap", y="y_umap", color="fixed.acidity")

> **Pregunta:** ¿Son suficientes los detectores de outliers sobre sola una variable?

## Métodos de Manejo de Outliers Multivariados


Este tipo de métodos permiten encontrar outliers considerando no solo un atributo en particular, si no que todos los atributos al mismo tiempo.

### Paréntesis: Clustering


**El Clustering es la tarea de agrupar distintas observaciones según su similitud.**

Es de tipo no-supervisado (no requiere etiquetas para entrenar).

![](./resources/clustering.png)

### DBScan

DBscan es un algoritmo de clustering basado en densidad. Su funcionamiento se basa en clasificar los datos en tres categorías:

- Core point: Es un punto que contiene un número minimo de vecinos cerca de un vecindario (esfera de radio ɛ)

- Border point: Es un punto que no es core, pero que es alcanzable por un Core Point. Es decir, existe un camino entre el Core Point y este.

- Outlier: Es un punto que no es core point y a la vez, que no tiene un camino entre un Core Point u Border Point.

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/dbscan.png" style="width: 50%;">
</div>

In [37]:
from sklearn.cluster import DBSCAN

# El parámetro eps indicará el tamaño de la esfera y por tanto, la cantidad de outliers
# Mientras menor sea el tamaño de eps, mayor la cantidad de outliers
clustering = DBSCAN(eps=0.3, min_samples=3).fit(scaled_features)

db_scan_labels = clustering.labels_

db_scan_labels

array([0, 0, 0, ..., 0, 0, 0])

In [38]:
db_scan_labels.max()

4

In [39]:
# Nota: Los labels de los outliers son -1
df[db_scan_labels == -1]

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,fixed.acidity_zscores,fixed.acidity_outlier,total.sulfur.dioxide_outlier,x_tsne,y_tsne,x_umap,y_umap
14,7.8,0.610,0.29,1.60,0.114,9.0,29.0,0.99740,3.26,1.56,9.1,5,-0.298547,False,False,1.619411,-15.833605,9.028958,2.153535
34,6.9,0.605,0.12,10.70,0.073,40.0,83.0,0.99930,3.45,0.52,9.4,6,-0.815625,False,False,19.547657,-43.049095,5.167263,4.070288
43,7.5,0.490,0.20,2.60,0.332,8.0,14.0,0.99680,3.21,0.90,10.5,6,-0.470907,False,False,1.216407,-21.795691,9.144739,2.151227
95,5.0,1.020,0.04,1.40,0.045,41.0,85.0,0.99380,3.75,0.48,10.5,4,-1.907233,False,False,-30.184364,24.848570,10.340993,7.680265
152,9.2,0.520,1.00,3.40,0.610,32.0,69.0,0.99960,2.74,2.00,9.4,4,0.505795,False,False,43.678345,-31.860191,4.917307,-0.089898
182,8.9,0.610,0.49,2.00,0.270,23.0,110.0,0.99720,3.12,1.02,9.3,5,0.333436,False,False,38.999664,-28.102495,5.083943,0.040494
227,8.9,0.590,0.50,2.00,0.337,27.0,81.0,0.99640,3.04,1.61,9.5,6,0.333436,False,False,40.288944,-32.065857,5.030029,-0.017656
259,7.7,0.410,0.76,1.80,0.611,8.0,45.0,0.99680,3.06,1.26,9.4,5,-0.356000,False,False,43.915855,-29.895741,4.903592,-0.088630
282,7.7,0.270,0.68,3.50,0.358,5.0,10.0,0.99720,3.25,1.08,9.9,7,-0.356000,False,False,43.065651,-27.816095,4.940451,-0.067383
325,10.0,0.490,0.20,11.00,0.071,13.0,50.0,1.00150,3.16,0.69,9.2,6,0.965420,False,False,25.955820,-41.645359,4.900012,4.187860


In [40]:
# Es más sencillo visualizar los outliers con color=db_scan_labels == -1
px.scatter(df, x="x_umap", y="y_umap", color=db_scan_labels == -1)

In [41]:
px.scatter(df, x="x_umap", y="y_umap", color=db_scan_labels.astype(str))

In [42]:
from sklearn.cluster import DBSCAN

# El parámetro eps indicará el tamaño de la esfera y por tanto, la cantidad de outliers
# Mientras menor sea el tamaño de eps, mayor la cantidad de outliers
clustering_projections = DBSCAN(eps=0.3, min_samples=3).fit(wine_features_umap_embedded)

db_scan_labels = clustering_projections.labels_

db_scan_labels

array([0, 0, 0, ..., 2, 2, 1])

In [43]:
px.scatter(df, x="x_umap", y="y_umap", color=db_scan_labels.astype(str))

## Métodos Específicos Provistos por Scikit-Learn

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/outlier_methods_sklearn.png" style="width: 50%;">
</div>

### Isolation Forest

El concepto subyacente de Isolation Forest es que las anomalías son datos que son pocos y diferentes, y por lo tanto, más fáciles de *aislar* que los datos normales. Isolar se refiere al proceso de separar un punto de datos del resto del conjunto de datos.

#### ¿Cómo funciona?

1. Construcción de Árboles de Aislamiento: Isolation Forest construye múltiples árboles de decisión, llamados árboles de aislamiento, para aislar cada muestra de datos. Para cada árbol, el proceso es como sigue:
    - Selecciona aleatoriamente una variable y luego selecciona un valor aleatorio de corte entre el mínimo y máximo valor de la característica seleccionada.
    - Divide el conjunto de datos en dos subconjuntos: uno con valores menores que el valor de corte y otro con valores mayores o iguales.
    - Repite este proceso recursivamente en cada subconjunto hasta que todos los datos han sido aislados o se alcanza un límite predeterminado en la profundidad del árbol.
2. Propiedad de Aislamiento: En teoría, las anomalías, al ser pocos y distintos, **requieren menos divisiones** aleatorias para ser aislados en comparación con los puntos de datos normales. Por lo tanto, los puntos de datos que tienen caminos cortos en los árboles son más propensos a ser anomalías.

#### Cálculo de puntuación de Anomalía

Una vez construidos los árboles, Isolation Forest evalúa la anormalidad de los datos utilizando una puntuación de anomalía basada en la **longitud del camino promedio** de un punto de datos en los árboles de aislamiento.
**Cuanto más corto es el camino promedio de un punto de datos, más alta es la puntuación de anomalía**, lo que indica una mayor probabilidad de ser una anomalía.

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/itree.png" style="width: 50%;">
</div>
    
<center>Fuente:https://betterprogramming.pub/anomaly-detection-with-isolation-forest-e41f1f55cc6</center>

#### Ventajas de Isolation Forest
- **Eficiencia**: Es eficaz en grandes conjuntos de datos y con una alta dimensionalidad de datos.
- **No necesita normalización**: A diferencia de otros métodos que requieren normalización de datos, Isolation Forest no necesita normalizar los datos porque el método de partición es independiente de la distribución.
- **Escalabilidad y Facilidad de Implementación**: Es relativamente fácil de implementar y escalar en comparación con otros métodos de detección de anomalías.


<center>Trafico web a través del tiempo<center/>

<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Anomalous_Web_Traffic.png" style="width: 70%;">
</div>


In [44]:
import numpy as np
from sklearn.ensemble import IsolationForest

isf = IsolationForest(n_estimators=20, random_state=42)
outliers = isf.fit_predict(scaled_features)

In [45]:
px.scatter(df, x="x_umap", y="y_umap", color=outliers.astype(str))

#### Novelty Detection

Ya que tenemos el modelo entrenado, podemos aplicarlo para clasificar una nueva observación (fuera del conjunto de entrenamiento) como outlier o no. Podemos lograr esto a través del método `.predict`:

In [46]:
row = scaled_features[100, :].copy() # obtenemos una fila
row

array([0.32743363, 0.33561644, 0.3       , 0.08219178, 0.12020033,
       0.14084507, 0.15547703, 0.52349486, 0.51968504, 0.16766467,
       0.27692308])

In [47]:
isf.predict([row]) # 1: no es outlier

array([1])

In [48]:
row

array([0.32743363, 0.33561644, 0.3       , 0.08219178, 0.12020033,
       0.14084507, 0.15547703, 0.52349486, 0.51968504, 0.16766467,
       0.27692308])

In [49]:
row[0] *= -100 # cambiar feature * -1000 
row[5] *= 500 # cambiar feature * 500
row[10] *= 999 # cambiar feature * 999
row

array([-3.27433628e+01,  3.35616438e-01,  3.00000000e-01,  8.21917808e-02,
        1.20200334e-01,  7.04225352e+01,  1.55477032e-01,  5.23494860e-01,
        5.19685039e-01,  1.67664671e-01,  2.76646154e+02])

In [50]:
isf.predict([row]) # -1: es outlier

array([-1])